# MIMIC‑IV on BigQuery

## Environment Bootstrap & Smoke Test

Purpose: make a clean, reproducible start on a new machine.

Outcome: verify auth, project config, and dataset access; provide a reusable BigQuery runner for the build notebook.

## 0. Prerequisites (one-time)

**Accounts & access**
- PhysioNet access to MIMIC‑IV on BigQuery; in BigQuery Console star project `physionet-data`.
- A Google Cloud **Project ID** with BigQuery API enabled (this is your **billing** project).

**CLI & environment**
- Google Cloud SDK (gcloud) installed and on PATH.
- Conda environment created from `environment.yml` and Jupyter kernel selected.
- A project‑local **`.env`** with the variables below.

**.env variables**
```ini
MIMIC_BACKEND=bigquery
WORK_PROJECT=mimic-hypercapnia #or another <your-billing-project-id>
BQ_PHYSIONET_PROJECT=physionet-data
BQ_DATASET_HOSP=mimiciv_3_1_hosp
BQ_DATASET_ICU=mimiciv_3_1_icu
BQ_DATASET_ED=mimiciv_ed
# GOOGLE_APPLICATION_CREDENTIALS=/Users/<you>/.config/gcloud/application_default_credentials.json
```

**Command line quickstart**
```bash
brew install --cask google-cloud-sdk
gcloud init
gcloud auth application-default login
gcloud services enable bigquery.googleapis.com --project <your-billing-project-id>
ls -l ~/.config/gcloud/application_default_credentials.json
```

## 1. Load configuration from `.env` and validate

In [1]:
import os
from pathlib import Path

try:
    from dotenv import load_dotenv
    for candidate in [".env", ".env.local"]:
        if Path(candidate).exists():
            load_dotenv(candidate, override=False)
            print(f"Loaded {candidate}")
except Exception as e:
    print("Note: python-dotenv not loaded:", e)

adc = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if adc:
    expanded = os.path.expanduser(adc)
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = expanded
    print("GOOGLE_APPLICATION_CREDENTIALS:", expanded, "(exists:", Path(expanded).exists(), ")")

WORK_PROJECT = os.getenv("WORK_PROJECT")
PHYS = os.getenv("BQ_PHYSIONET_PROJECT", "physionet-data")
HOSP = os.getenv("BQ_DATASET_HOSP", "mimiciv_3_1_hosp")
ICU  = os.getenv("BQ_DATASET_ICU",  "mimiciv_3_1_icu")
ED   = os.getenv("BQ_DATASET_ED",   "mimiciv_ed")

print("WORK_PROJECT:", WORK_PROJECT)
print("PHYSIONET project:", PHYS)
print("Datasets -> HOSP:", HOSP, "| ICU:", ICU, "| ED:", ED)

if WORK_PROJECT in (None, "", "your-gcp-project-id", "YOUR_GCP_PROJECT_ID"):
    raise RuntimeError("Set WORK_PROJECT in .env to your actual GCP Project ID (not the display name).")


Loaded .env
WORK_PROJECT: mimic-hypercapnia
PHYSIONET project: physionet-data
Datasets -> HOSP: mimiciv_3_1_hosp | ICU: mimiciv_3_1_icu | ED: mimiciv_ed


## 2. Optional: silence gRPC ALTS warnings

The ALTS messages you may see are harmless:
```
ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
```
Silence them for this kernel:

In [2]:
import os
os.environ["GRPC_VERBOSITY"] = "NONE"
os.environ["GRPC_LOG_SEVERITY_LEVEL"] = "ERROR"
print("gRPC verbosity disabled for this process.")


gRPC verbosity disabled for this process.


## 3. Create BigQuery client and run a trivial query

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client(project=WORK_PROJECT)
print("BigQuery client project:", client.project)
smoke = client.query("SELECT 1 AS ok").result().to_dataframe()
smoke


BigQuery client project: mimic-hypercapnia


,ok
0,1


## 4. Dataset checks — counts and peeks

Use the dataset names exactly as shown in Explorer.

In [4]:
sql1 = f"SELECT COUNT(*) AS n FROM `{PHYS}.{HOSP}.admissions`"
sql2 = f"SELECT COUNT(*) AS n FROM `{PHYS}.{ICU}.icustays`"
df1 = client.query(sql1).result().to_dataframe()
df2 = client.query(sql2).result().to_dataframe()
print(sql1) ; display(df1)
print(sql2) ; display(df2)


SELECT COUNT(*) AS n FROM `physionet-data.mimiciv_3_1_hosp.admissions`


,n
0,546028


SELECT COUNT(*) AS n FROM `physionet-data.mimiciv_3_1_icu.icustays`


,n
0,94458


In [5]:
sql_pat = f"SELECT * FROM `{PHYS}.{HOSP}.patients` LIMIT 5"
client.query(sql_pat).result().to_dataframe()


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10078138,F,18,2110,2017 - 2019,NaT
1,10180372,M,18,2110,2008 - 2010,NaT
2,10686175,M,18,2110,2011 - 2013,NaT
3,10851602,F,18,2110,2014 - 2016,NaT
4,10902424,F,18,2110,2017 - 2019,NaT


In [6]:
try:
    sql_ed = f"SELECT COUNT(*) AS n FROM `{PHYS}.{ED}.edstays`"
    df_ed = client.query(sql_ed).result().to_dataframe()
    print(sql_ed) ; display(df_ed)
except Exception as e:
    print("ED dataset check skipped or failed:", e)


SELECT COUNT(*) AS n FROM `physionet-data.mimiciv_ed.edstays`


,n
0,425087


## 5. Reusable runner

In [7]:
def run_sql_bq(sql: str):
    job = client.query(sql)
    return job.result().to_dataframe()

run_sql_bq(f"SELECT COUNT(*) AS n FROM `{PHYS}.{HOSP}.admissions`")


,n
0,546028


## 6. Troubleshooting quicklist

- **ProjectId must be non-empty** → Set `WORK_PROJECT` in `.env`; enable BigQuery API.
- **Access Denied** → Use dataset names from Explorer; confirm approved account; query billing project is yours.
- **ADC file not found** → `gcloud auth application-default login`; avoid `~` unless expanded.
- **ALTS warnings** → harmless; silenced by env vars above.
- **Kernel/env mismatch** → `pip show google-cloud-bigquery` must succeed in this kernel.

## Code to recreate the MIMIC tabular dataset

**Goal**: Reproduce the legacy `MIMIC_hypercap.xlsx` using BigQuery (no Summit).

**Inputs**: Access to `physionet-data` BigQuery project and datasets `mimiciv_3_1_hosp`, `mimiciv_3_1_icu`, and your ED dataset.  (`mimiciv_ed`)

**Output**: A single wide table with ICD flags, demographics, ED triage, outcomes, and first blood gas values.

### 1. Configuration and BigQuery runner

In [ ]:
import os
from pathlib import Path
import pandas as pd
from google.cloud import bigquery

# Optional: silence gRPC ALTS warnings (harmless otherwise)
os.environ.setdefault("GRPC_VERBOSITY", "NONE")
os.environ.setdefault("GRPC_LOG_SEVERITY_LEVEL", "ERROR")

# Load .env
try:
    from dotenv import load_dotenv
    for candidate in [".env", ".env.local"]:
        if Path(candidate).exists():
            load_dotenv(candidate, override=False)
            print(f"Loaded {candidate}")
except Exception as e:
    print("Note: python-dotenv not loaded:", e)

# Expand '~' in ADC if present
adc = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if adc:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser(adc)

WORK_PROJECT = os.getenv("WORK_PROJECT")
PHYS = os.getenv("BQ_PHYSIONET_PROJECT", "physionet-data")
HOSP = os.getenv("BQ_DATASET_HOSP", "mimiciv_3_1_hosp")
ICU  = os.getenv("BQ_DATASET_ICU",  "mimiciv_3_1_icu")
ED   = os.getenv("BQ_DATASET_ED",   "mimiciv_ed")

assert WORK_PROJECT not in (None, "", "your-gcp-project-id", "YOUR_GCP_PROJECT_ID"), "Set WORK_PROJECT to your billing project id"
print("Billing project:", WORK_PROJECT)
print("Datasets ->", {"HOSP": HOSP, "ICU": ICU, "ED": ED})

client = bigquery.Client(project=WORK_PROJECT)

def run_sql_bq(sql: str, params: dict | None = None) -> pd.DataFrame:
    job_config = bigquery.QueryJobConfig()
    bq_params = []
    if params:
        for name, val in params.items():
            if isinstance(val, list):
                # infer list type as INT64 if contents are ints, else STRING
                if all(isinstance(x, (int,)) or (isinstance(x, float) and float(x).is_integer()) for x in val if x is not None):
                    bq_params.append(bigquery.ArrayQueryParameter(name, "INT64", [int(x) for x in val if x is not None]))
                else:
                    bq_params.append(bigquery.ArrayQueryParameter(name, "STRING", [None if x is None else str(x) for x in val]))
            else:
                # scalar
                if isinstance(val, (int,)) or (isinstance(val, float) and float(val).is_integer()):
                    bq_params.append(bigquery.ScalarQueryParameter(name, "INT64", int(val)))
                else:
                    bq_params.append(bigquery.ScalarQueryParameter(name, "STRING", str(val)))
        job_config.query_parameters = bq_params
    job = client.query(sql, job_config=job_config)
    return job.result().to_dataframe(create_bqstorage_client=True)

def require_hadm(df: pd.DataFrame, name="table"):
    if "hadm_id" in df.columns:
        return df
    if df.index.name == "hadm_id":
        return df.reset_index()
    raise KeyError(f"{name} has no 'hadm_id'. Ensure you're operating on the admission-level table.")

Loaded .env
Billing project: mimic-hypercapnia
Datasets -> {'HOSP': 'mimiciv_3_1_hosp', 'ICU': 'mimiciv_3_1_icu', 'ED': 'mimiciv_ed'}


### 2. Cohort by ICD codes (ED diagnosis + Inpatient diagnoses)

In [9]:
# ICD lists (dotless, uppercase)
ICD10_CODES = ['J9602','J9612','J9622','J9692','E662']
ICD9_CODES  = ['27803']

cohort_sql = f"""
-- Target hypercapnic respiratory failure codes
WITH target_codes AS (
  SELECT 'J9602' AS code, 10 AS ver UNION ALL
  SELECT 'J9612', 10 UNION ALL
  SELECT 'J9622', 10 UNION ALL
  SELECT 'J9692', 10 UNION ALL
  SELECT 'E662',  10 UNION ALL
  SELECT '27803', 9
),

-- Hospital diagnoses restricted to target codes
hosp_dx AS (
  SELECT
    d.subject_id,
    d.hadm_id,
    UPPER(REPLACE(d.icd_code, '.', '')) AS code_norm,
    d.icd_version
  FROM `{PHYS}.{HOSP}.diagnoses_icd` d
  JOIN target_codes t
    ON t.ver = d.icd_version
   AND t.code = UPPER(REPLACE(d.icd_code, '.', ''))
  WHERE d.hadm_id IS NOT NULL
),

hosp_flags AS (
  SELECT
    subject_id, hadm_id,
    MAX(IF(icd_version=10 AND code_norm='J9602',1,0)) AS ICD10_J9602,
    MAX(IF(icd_version=10 AND code_norm='J9612',1,0)) AS ICD10_J9612,
    MAX(IF(icd_version=10 AND code_norm='J9622',1,0)) AS ICD10_J9622,
    MAX(IF(icd_version=10 AND code_norm='J9692',1,0)) AS ICD10_J9692,
    MAX(IF(icd_version=10 AND code_norm='E662', 1,0)) AS ICD10_E662,
    MAX(IF(icd_version=9  AND code_norm='27803',1,0)) AS ICD9_27803
  FROM hosp_dx
  GROUP BY subject_id, hadm_id
),

-- ED diagnoses restricted to target codes, mapped to the admission
ed_dx AS (
  SELECT
    s.subject_id,
    s.hadm_id,
    s.stay_id,
    s.intime AS ed_intime,
    UPPER(REPLACE(d.icd_code, '.', '')) AS code_norm,
    d.icd_version
  FROM `{PHYS}.{ED}.diagnosis` d
  JOIN `{PHYS}.{ED}.edstays` s
    ON s.subject_id = d.subject_id AND s.stay_id = d.stay_id
  JOIN target_codes t
    ON t.ver = d.icd_version
   AND t.code = UPPER(REPLACE(d.icd_code, '.', ''))
  WHERE s.hadm_id IS NOT NULL
),

-- Flags per ED stay (if multiple ED stays map to the same HADM, keep flags for each)
ed_flags_by_stay AS (
  SELECT
    subject_id, hadm_id, stay_id, MIN(ed_intime) AS ed_intime,
    MAX(IF(icd_version=10 AND code_norm='J9602',1,0)) AS ICD10_J9602,
    MAX(IF(icd_version=10 AND code_norm='J9612',1,0)) AS ICD10_J9612,
    MAX(IF(icd_version=10 AND code_norm='J9622',1,0)) AS ICD10_J9622,
    MAX(IF(icd_version=10 AND code_norm='J9692',1,0)) AS ICD10_J9692,
    MAX(IF(icd_version=10 AND code_norm='E662', 1,0)) AS ICD10_E662,
    MAX(IF(icd_version=9  AND code_norm='27803',1,0)) AS ICD9_27803
  FROM ed_dx
  GROUP BY subject_id, hadm_id, stay_id
),

-- Collapse to one ED stay per admission (earliest intime) and OR the flags
ed_by_hadm AS (
  SELECT
    subject_id,
    hadm_id,
    (ARRAY_AGG(stay_id ORDER BY ed_intime LIMIT 1))[OFFSET(0)] AS stay_id,
    MAX(ICD10_J9602) AS ICD10_J9602,
    MAX(ICD10_J9612) AS ICD10_J9612,
    MAX(ICD10_J9622) AS ICD10_J9622,
    MAX(ICD10_J9692) AS ICD10_J9692,
    MAX(ICD10_E662 ) AS ICD10_E662,
    MAX(ICD9_27803) AS ICD9_27803
  FROM ed_flags_by_stay
  GROUP BY subject_id, hadm_id
),

-- Combine ED and hospital flags at the admission level
combined AS (
  SELECT
    COALESCE(h.subject_id, e.subject_id) AS subject_id,
    e.stay_id,  -- ED stay leading to the admission (if any)
    COALESCE(h.hadm_id, e.hadm_id) AS hadm_id,
    GREATEST(IFNULL(h.ICD10_J9602,0), IFNULL(e.ICD10_J9602,0)) AS ICD10_J9602,
    GREATEST(IFNULL(h.ICD10_J9612,0), IFNULL(e.ICD10_J9612,0)) AS ICD10_J9612,
    GREATEST(IFNULL(h.ICD10_J9622,0), IFNULL(e.ICD10_J9622,0)) AS ICD10_J9622,
    GREATEST(IFNULL(h.ICD10_J9692,0), IFNULL(e.ICD10_J9692,0)) AS ICD10_J9692,
    GREATEST(IFNULL(h.ICD10_E662 ,0), IFNULL(e.ICD10_E662 ,0)) AS ICD10_E662,
    GREATEST(IFNULL(h.ICD9_27803,0), IFNULL(e.ICD9_27803,0)) AS ICD9_27803
  FROM hosp_flags h
  FULL OUTER JOIN ed_by_hadm e
    ON h.hadm_id = e.hadm_id
)

SELECT
  subject_id, stay_id, hadm_id,
  ICD10_J9602, ICD10_J9612, ICD10_J9622, ICD10_J9692, ICD10_E662, ICD9_27803,
  IF((ICD10_J9602+ICD10_J9612+ICD10_J9622+ICD10_J9692+ICD10_E662+ICD9_27803) > 0, 1, 0) AS any_hypercap_icd
FROM combined
WHERE (ICD10_J9602+ICD10_J9612+ICD10_J9622+ICD10_J9692+ICD10_E662+ICD9_27803) > 0
"""
cohort = run_sql_bq(cohort_sql)
cohort.head()

flags = ["ICD10_J9602","ICD10_J9612","ICD10_J9622","ICD10_J9692","ICD10_E662","ICD9_27803"]

# 1) any_hypercap_icd must imply at least one component flag
bad = cohort[(cohort["any_hypercap_icd"] == 1) & (cohort[flags].sum(axis=1) == 0)]
assert bad.empty, f"Inconsistent flags: {len(bad)} rows have any=1 but all component flags=0"

# 2) one line per admission
assert cohort["hadm_id"].isna().sum() == 0, "Cohort produced rows without hadm_id"
assert cohort["hadm_id"].nunique() == len(cohort), "Cohort has duplicate hadm_id rows"

# 3) recompute hadm_list for downstream lab extraction
hadm_list = sorted(cohort["hadm_id"].dropna().unique().tolist())

note: """If physionet-data.mimiciv_derived.bg is available in your region/project, you can source the Lab pairs from there (already specimen‑pivoted) and keep only the ICU POC branch from above. The MIMIC code and SOFA concept reference this table.""" - [ ] TODO: look into this

In [ ]:
params = {"hadms": hadm_list}

bg_pairs_sql = f"""
WITH hadms AS (SELECT hadm_id FROM UNNEST(@hadms) AS hadm_id),

/* ---------------- LAB (HOSP) ---------------- */
hosp_cand AS (
  SELECT
    le.subject_id, le.hadm_id, le.charttime, le.specimen_id,
    CAST(le.valuenum AS FLOAT64) AS val, LOWER(COALESCE(le.valueuom,'')) AS uom,
    LOWER(di.label) AS lbl, LOWER(COALESCE(di.fluid,'')) AS fl, LOWER(COALESCE(di.category,'')) AS cat
  FROM `{PHYS}.{HOSP}.labevents`  le
  JOIN `{PHYS}.{HOSP}.d_labitems` di ON di.itemid = le.itemid
  JOIN hadms h ON h.hadm_id = le.hadm_id
  WHERE le.valuenum IS NOT NULL
    AND (cat LIKE '%blood gas%' OR lbl LIKE '%pco2%' OR REGEXP_CONTAINS(lbl, r'\\bph\\b'))
    AND (
          REGEXP_CONTAINS(lbl, r'\\bph\\b')
       OR REGEXP_CONTAINS(lbl, r'\\bpa?\\s*co(?:2|₂)\\b')
       OR lbl LIKE '%pco2%' OR lbl LIKE '%paco2%'
        )
    AND NOT REGEXP_CONTAINS(lbl, r'(tco2|total|content|bicar|etco2|end[- ]?tidal)')
),
hosp_spec AS (
  SELECT le.specimen_id, LOWER(COALESCE(le.value,'')) AS spec_val
  FROM `{PHYS}.{HOSP}.labevents` le
  JOIN `{PHYS}.{HOSP}.d_labitems` di ON di.itemid = le.itemid
  WHERE le.specimen_id IS NOT NULL AND REGEXP_CONTAINS(LOWER(di.label), r'(specimen|sample)')
),
hosp_class AS (
  SELECT
    c.subject_id, c.hadm_id, c.charttime, c.specimen_id, c.val, c.uom, c.lbl, c.fl,
    CASE
      WHEN REGEXP_CONTAINS(c.lbl, r'\\bph\\b') THEN 'ph'
      WHEN (c.lbl LIKE '%pco2%' OR REGEXP_CONTAINS(c.lbl, r'\\bpa?\\s*co(?:2|₂)\\b')) THEN 'pco2'
      ELSE NULL
    END AS analyte,
    CASE
      WHEN REGEXP_CONTAINS(s.spec_val, r'arter') OR REGEXP_CONTAINS(s.spec_val, r'\\bart\\b') THEN 'arterial'
      WHEN REGEXP_CONTAINS(s.spec_val, r'ven|mixed|central') THEN 'venous'
      WHEN c.fl LIKE '%arterial%' OR REGEXP_CONTAINS(c.lbl, r'\\b(abg|art|arterial|a[- ]?line)\\b') THEN 'arterial'
      WHEN c.fl LIKE '%ven%'      OR REGEXP_CONTAINS(c.lbl, r'\\b(vbg|ven|venous|mixed|central)\\b') THEN 'venous'
      ELSE NULL
    END AS site
  FROM hosp_cand c
  LEFT JOIN hosp_spec s USING (specimen_id)
),
hosp_pairs AS (
  SELECT
    hadm_id, specimen_id,
    MIN(charttime) AS sample_time,
    MAX(IF(analyte='ph',   val, NULL)) AS ph,
    MAX(IF(analyte='pco2', val, NULL)) AS pco2_raw,
    (ARRAY_AGG(IF(analyte='pco2', uom, NULL) IGNORE NULLS LIMIT 1))[OFFSET(0)] AS pco2_uom,
    (ARRAY_AGG(IF(analyte='ph',   uom, NULL) IGNORE NULLS LIMIT 1))[OFFSET(0)] AS ph_uom,
    (ARRAY_AGG(site IGNORE NULLS LIMIT 1))[OFFSET(0)] AS site
  FROM hosp_class
  GROUP BY hadm_id, specimen_id
  HAVING ph IS NOT NULL AND pco2_raw IS NOT NULL AND site IN ('arterial','venous')
),
hosp_pairs_std AS (
  SELECT
    hadm_id, specimen_id, sample_time, site,
    ph, ph_uom,
    CASE WHEN pco2_uom = 'kpa' THEN pco2_raw * 7.50062 ELSE pco2_raw END AS pco2_mmHg,
    COALESCE(NULLIF(pco2_uom,''),'mmhg') AS pco2_uom_norm
  FROM hosp_pairs
  WHERE ph BETWEEN 6.3 AND 7.8 AND (CASE WHEN pco2_uom='kpa' THEN pco2_raw*7.50062 ELSE pco2_raw END) BETWEEN 5 AND 200
),
lab_abg AS (
  SELECT hadm_id,
         ph           AS lab_abg_ph,
         ph_uom       AS lab_abg_ph_uom,
         pco2_mmHg    AS lab_abg_paco2,
         pco2_uom_norm AS lab_abg_paco2_uom,
         sample_time  AS lab_abg_time
  FROM (
     SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
     FROM hosp_pairs_std WHERE site='arterial'
  ) WHERE rn=1
),
lab_vbg AS (
  SELECT hadm_id,
         ph           AS lab_vbg_ph,
         ph_uom       AS lab_vbg_ph_uom,
         pco2_mmHg    AS lab_vbg_paco2,
         pco2_uom_norm AS lab_vbg_paco2_uom,
         sample_time  AS lab_vbg_time
  FROM (
     SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
     FROM hosp_pairs_std WHERE site='venous'
  ) WHERE rn=1
),

/* ---------------- POC (ICU) ---------------- */
icu_cand AS (
  SELECT
    ie.hadm_id, ce.stay_id, ce.charttime,
    CAST(ce.valuenum AS FLOAT64) AS val, LOWER(COALESCE(ce.valueuom,'')) AS uom,
    LOWER(di.label) AS lbl
  FROM `{PHYS}.{ICU}.chartevents` ce
  JOIN `{PHYS}.{ICU}.d_items`  di ON di.itemid = ce.itemid
  JOIN `{PHYS}.{ICU}.icustays` ie ON ie.stay_id = ce.stay_id
  JOIN hadms h ON h.hadm_id = ie.hadm_id
  WHERE ce.valuenum IS NOT NULL
    AND (
          REGEXP_CONTAINS(lbl, r'\\bph\\b')
       OR REGEXP_CONTAINS(lbl, r'\\bpa?\\s*co(?:2|₂)\\b')
       OR lbl LIKE '%pco2%' OR lbl LIKE '%paco2%'
        )
    AND NOT REGEXP_CONTAINS(lbl, r'(tco2|total|content|bicar|etco2|end[- ]?tidal)')
),
icu_class AS (
  SELECT
    hadm_id, stay_id, charttime, val, uom, lbl,
    CASE
      WHEN REGEXP_CONTAINS(lbl, r'\\bph\\b') THEN 'ph'
      WHEN (lbl LIKE '%pco2%' OR REGEXP_CONTAINS(lbl, r'\\bpa?\\s*co(?:2|₂)\\b')) THEN 'pco2'
      ELSE NULL
    END AS analyte,
    CASE
      WHEN REGEXP_CONTAINS(lbl, r'\\b(abg|art|arterial|a[- ]?line)\\b') THEN 'arterial'
      WHEN REGEXP_CONTAINS(lbl, r'\\b(vbg|ven|venous|mixed|central)\\b') THEN 'venous'
      ELSE NULL
    END AS site
  FROM icu_cand
),
icu_pairs AS (
  SELECT
    hadm_id, stay_id, charttime AS sample_time,
    MAX(IF(analyte='ph',   val, NULL)) AS ph,
    MAX(IF(analyte='pco2', val, NULL)) AS pco2_raw,
    (ARRAY_AGG(IF(analyte='pco2', uom, NULL) IGNORE NULLS LIMIT 1))[OFFSET(0)] AS pco2_uom,
    (ARRAY_AGG(IF(analyte='ph',   uom, NULL) IGNORE NULLS LIMIT 1))[OFFSET(0)] AS ph_uom,
    (ARRAY_AGG(site IGNORE NULLS LIMIT 1))[OFFSET(0)] AS site
  FROM icu_class
  GROUP BY hadm_id, stay_id, sample_time
  HAVING ph IS NOT NULL AND pco2_raw IS NOT NULL AND site IN ('arterial','venous')
),
icu_pairs_std AS (
  SELECT
    hadm_id, stay_id, sample_time, site,
    ph, ph_uom,
    CASE WHEN pco2_uom='kpa' THEN pco2_raw*7.50062 ELSE pco2_raw END AS pco2_mmHg,
    COALESCE(NULLIF(pco2_uom,''),'mmhg') AS pco2_uom_norm
  FROM icu_pairs
  WHERE ph BETWEEN 6.3 AND 7.8 AND (CASE WHEN pco2_uom='kpa' THEN pco2_raw*7.50062 ELSE pco2_raw END) BETWEEN 5 AND 200
),
poc_abg AS (
  SELECT hadm_id,
         ph           AS poc_abg_ph,
         ph_uom       AS poc_abg_ph_uom,
         pco2_mmHg    AS poc_abg_paco2,
         pco2_uom_norm AS poc_abg_paco2_uom,
         sample_time  AS poc_abg_time
  FROM (
     SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
     FROM icu_pairs_std WHERE site='arterial'
  ) WHERE rn=1
),
poc_vbg AS (
  SELECT hadm_id,
         ph           AS poc_vbg_ph,
         ph_uom       AS poc_vbg_ph_uom,
         pco2_mmHg    AS poc_vbg_paco2,
         pco2_uom_norm AS poc_vbg_paco2_uom,
         sample_time  AS poc_vbg_time
  FROM (
     SELECT *, ROW_NUMBER() OVER (PARTITION BY hadm_id ORDER BY sample_time) rn
     FROM icu_pairs_std WHERE site='venous'
  ) WHERE rn=1
)

/* --------------- Final one-row-per hadm --------------- */
SELECT
  h.hadm_id,
  la.lab_abg_ph, la.lab_abg_ph_uom, la.lab_abg_paco2, la.lab_abg_paco2_uom, la.lab_abg_time,
  lv.lab_vbg_ph, lv.lab_vbg_ph_uom, lv.lab_vbg_paco2, lv.lab_vbg_paco2_uom, lv.lab_vbg_time,
  pa.poc_abg_ph, pa.poc_abg_ph_uom, pa.poc_abg_paco2, pa.poc_abg_paco2_uom, pa.poc_abg_time,
  pv.poc_vbg_ph, pv.poc_vbg_ph_uom, pv.poc_vbg_paco2, pv.poc_vbg_paco2_uom, pv.poc_vbg_time
FROM hadms h
LEFT JOIN lab_abg la USING (hadm_id)
LEFT JOIN lab_vbg lv USING (hadm_id)
LEFT JOIN poc_abg pa USING (hadm_id)
LEFT JOIN poc_vbg pv USING (hadm_id)
"""
bg_pairs = run_sql_bq(bg_pairs_sql, params)
bg_pairs.head()

In [ ]:
# Map to legacy names (keep LAB-VBG as extra columns if you want them)
rename_map = {
    # LAB ABG -> legacy "lab_*"
    "lab_abg_paco2":      "lab_paco2",
    "lab_abg_paco2_uom":  "lab_paco2_uom",
    "lab_abg_time":       "abg_time",
    "lab_abg_ph":         "lab_ph",
    "lab_abg_ph_uom":     "lab_ph_uom",

    # POC ABG -> legacy "poc_*"
    "poc_abg_paco2":      "poc_paco2",
    "poc_abg_paco2_uom":  "poc_paco2_uom",
    "poc_abg_time":       "poc_paco2_time",
    "poc_abg_ph":         "poc_ph",
    "poc_abg_ph_uom":     "poc_ph_uom",

    # POC VBG -> legacy "poc_vbg_*" already matches; no rename needed
}
bg_pairs_legacy = bg_pairs.rename(columns=rename_map).copy()
bg_pairs_legacy["poc_ph_time"] = bg_pairs_legacy["poc_paco2_time"]  # co-time for POC ABG analytes

# Merge into master admission-level df
assert "hadm_id" in df.columns, "df must have 'hadm_id' before merge"
df = df.merge(bg_pairs_legacy, on="hadm_id", how="left")

# Sanity checks: ranges (fail fast if wrong analyte lands in wrong column)
def in_range(s, lo, hi): 
    s = pd.to_numeric(s, errors="coerce")
    return s.notna() & (s >= lo) & (s <= hi)

ph_cols  = [c for c in ["lab_ph","poc_ph","poc_vbg_ph","lab_vbg_ph"] if c in df.columns]
co2_cols = [c for c in ["lab_paco2","poc_paco2","poc_vbg_paco2","lab_vbg_paco2"] if c in df.columns]

for c in ph_cols:
    bad = (~in_range(df[c], 6.3, 7.8)).sum()
    print(f"{c} out-of-range (should be 6.3–7.8):", bad)

for c in co2_cols:
    bad = (~in_range(df[c], 5, 200)).sum()
    print(f"{c} out-of-range (should be 5–200 mmHg):", bad)

In [ ]:
# Ensure the master table really has hadm_id
if "hadm_id" not in df.columns:
    if df.index.name == "hadm_id":
        df = df.reset_index()
    else:
        raise KeyError("df has no 'hadm_id'. Make sure you're merging into the admission-level table.")

# Also ensure bg_pairs exists and has hadm_id before merge
assert "hadm_id" in bg_pairs.columns, "bg_pairs is missing 'hadm_id' – check the SQL cell."

KeyError: "df has no 'hadm_id'. Make sure you're merging into the admission-level table."

The ones below this might be superfulous now. 

### Save

In [ ]:
from datetime import datetime
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_base = f"MIMIC_hypercap_bq_abg_vbg_{ts}"

df.to_parquet(out_base + ".parquet", index=False)
try:
    import openpyxl  # noqa
    df.to_excel(out_base + ".xlsx", index=False)
except Exception as e:
    print("Excel write skipped (install openpyxl to enable):", e)

print("Wrote:", out_base + ".parquet")
print("Wrote:", out_base + ".xlsx")